In [1]:
%matplotlib inline

In [2]:
from matplotlib import pyplot as plt
import numpy as np
import os
from PIL import Image
import tensorflow as tf
import math
from skimage.transform import resize

In [3]:
tf.__version__

'1.0.0'

In [4]:
!bazel build -c opt //im2txt/...

Loading: 
Loading: 0 packages loaded
INFO: Analysed 17 targets (0 packages loaded).
Building: no action
INFO: Found 17 targets...
Building: no action
[0 / 1] [-----] BazelWorkspaceStatusAction stable-status.txt
INFO: Elapsed time: 0.174s, Critical Path: 0.01s
[1 / 1] no action
INFO: Build completed successfully, 1 total action


In [5]:
import sys
import collections

In [6]:
sys.path.insert(0, 'bazel-bin/im2txt/run_inference.runfiles/im2txt/')

In [7]:
from __future__ import absolute_import
from im2txt import configuration
from im2txt.inference_utils import caption_generator
from im2txt.inference_utils import vocabulary
from im2txt import show_and_tell_model
from im2txt.inference_utils import inference_wrapper_base

In [8]:
tf.logging.set_verbosity(tf.logging.INFO)

In [9]:
checkpoint_path = '/hdd/models/im2txt_2016_10_05.1000000/model.ckpt-1000000'

In [10]:
class InferenceWrapper(inference_wrapper_base.InferenceWrapperBase):
    """Model wrapper class for performing inference with a ShowAndTellModel."""

    def __init__(self):
        super(InferenceWrapper, self).__init__()
        self.fetches = {}
        self.st_custom_getter = None
        self.post_feed_image = lambda: result 
    
    def build_model(self, model_config):
        model = show_and_tell_model.ShowAndTellModel(model_config, mode="inference")
        model.st_custom_getter = self.st_custom_getter
        model.build()
        return model
    
    def _create_restore_fn(self, checkpoint_path, saver):
        def _resore(sess):
            pass
        return _resore
    
    def feed_image(self, sess, encoded_image):
        self.fetches['initial_state'] = tf.get_default_graph().get_tensor_by_name('lstm/initial_state:0')
        result = sess.run(fetches=self.fetches,
                                 feed_dict={"image_feed:0": encoded_image})
        self.post_feed_image(result)
        return result['initial_state']

    def inference_step(self, sess, input_feed, state_feed):
        softmax_output, state_output = sess.run(
            fetches=["softmax:0", "lstm/state:0"],
            feed_dict={
                "input_feed:0": input_feed,
                "lstm/state_feed:0": state_feed,
            })
        return softmax_output, state_output, None


In [15]:
model = InferenceWrapper()
config = configuration.ModelConfig()
reader = tf.train.NewCheckpointReader(checkpoint_path)

name_map = {
'lstm/basic_lstm_cell/weights': 'lstm/BasicLSTMCell/Linear/Matrix',
'lstm/basic_lstm_cell/biases': 'lstm/BasicLSTMCell/Linear/Bias'
}


def _create_restore_fn(self, checkpoint_path, saver):
    print "Someting"
st_variables_map = {}

def st_custom_getter(getter, name, shape, *args, **kwargs):
    if name in st_variables_map:
        return st_variables_map[name]
    print('Restoring', name, shape)
    kwargs['trainable'] = False
    c = None
    name = name_map.get(name) or name
    if reader.has_tensor(name):
        c = reader.get_tensor(name)
    elif reader.has_tensor('InceptionV3/' + name):
        c = reader.get_tensor('InceptionV3/' + name)
    assert(c is not None)
    
    kwargs['initializer'] = tf.constant_initializer(c)

    
    v = getter(name, shape, *args, **kwargs)
    st_variables_map[name]= v
    return v

st_variables = list(st_variables_map.values())

model.st_custom_getter = st_custom_getter

restore_fn = model.build_graph_from_config(config, checkpoint_path)
tf.trainable_variables()

INFO:tensorflow:Building model.
('Restoring', 'Conv2d_1a_3x3/weights', (3, 3, Dimension(3), 32))
('Restoring', 'Conv2d_1a_3x3/BatchNorm/beta', (Dimension(32),))
('Restoring', 'Conv2d_1a_3x3/BatchNorm/moving_mean', (Dimension(32),))
('Restoring', 'Conv2d_1a_3x3/BatchNorm/moving_variance', (Dimension(32),))
('Restoring', 'Conv2d_2a_3x3/weights', (3, 3, Dimension(32), 32))
('Restoring', 'Conv2d_2a_3x3/BatchNorm/beta', (Dimension(32),))
('Restoring', 'Conv2d_2a_3x3/BatchNorm/moving_mean', (Dimension(32),))
('Restoring', 'Conv2d_2a_3x3/BatchNorm/moving_variance', (Dimension(32),))
('Restoring', 'Conv2d_2b_3x3/weights', (3, 3, Dimension(32), 64))
('Restoring', 'Conv2d_2b_3x3/BatchNorm/beta', (Dimension(64),))
('Restoring', 'Conv2d_2b_3x3/BatchNorm/moving_mean', (Dimension(64),))
('Restoring', 'Conv2d_2b_3x3/BatchNorm/moving_variance', (Dimension(64),))
('Restoring', 'Conv2d_3b_1x1/weights', (1, 1, Dimension(64), 80))
('Restoring', 'Conv2d_3b_1x1/BatchNorm/beta', (Dimension(80),))
('Restoring

('Restoring', 'Mixed_6b/Branch_1/Conv2d_0a_1x1/weights', (1, 1, Dimension(768), 128))
('Restoring', 'Mixed_6b/Branch_1/Conv2d_0a_1x1/BatchNorm/beta', (Dimension(128),))
('Restoring', 'Mixed_6b/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_mean', (Dimension(128),))
('Restoring', 'Mixed_6b/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_variance', (Dimension(128),))
('Restoring', 'Mixed_6b/Branch_1/Conv2d_0b_1x7/weights', (1, 7, Dimension(128), 128))
('Restoring', 'Mixed_6b/Branch_1/Conv2d_0b_1x7/BatchNorm/beta', (Dimension(128),))
('Restoring', 'Mixed_6b/Branch_1/Conv2d_0b_1x7/BatchNorm/moving_mean', (Dimension(128),))
('Restoring', 'Mixed_6b/Branch_1/Conv2d_0b_1x7/BatchNorm/moving_variance', (Dimension(128),))
('Restoring', 'Mixed_6b/Branch_1/Conv2d_0c_7x1/weights', (7, 1, Dimension(128), 192))
('Restoring', 'Mixed_6b/Branch_1/Conv2d_0c_7x1/BatchNorm/beta', (Dimension(192),))
('Restoring', 'Mixed_6b/Branch_1/Conv2d_0c_7x1/BatchNorm/moving_mean', (Dimension(192),))
('Restoring', 'Mixed_6b/Branch_1/Co

('Restoring', 'Mixed_6e/Branch_2/Conv2d_0b_7x1/weights', (7, 1, Dimension(192), 192))
('Restoring', 'Mixed_6e/Branch_2/Conv2d_0b_7x1/BatchNorm/beta', (Dimension(192),))
('Restoring', 'Mixed_6e/Branch_2/Conv2d_0b_7x1/BatchNorm/moving_mean', (Dimension(192),))
('Restoring', 'Mixed_6e/Branch_2/Conv2d_0b_7x1/BatchNorm/moving_variance', (Dimension(192),))
('Restoring', 'Mixed_6e/Branch_2/Conv2d_0c_1x7/weights', (1, 7, Dimension(192), 192))
('Restoring', 'Mixed_6e/Branch_2/Conv2d_0c_1x7/BatchNorm/beta', (Dimension(192),))
('Restoring', 'Mixed_6e/Branch_2/Conv2d_0c_1x7/BatchNorm/moving_mean', (Dimension(192),))
('Restoring', 'Mixed_6e/Branch_2/Conv2d_0c_1x7/BatchNorm/moving_variance', (Dimension(192),))
('Restoring', 'Mixed_6e/Branch_2/Conv2d_0d_7x1/weights', (7, 1, Dimension(192), 192))
('Restoring', 'Mixed_6e/Branch_2/Conv2d_0d_7x1/BatchNorm/beta', (Dimension(192),))
('Restoring', 'Mixed_6e/Branch_2/Conv2d_0d_7x1/BatchNorm/moving_mean', (Dimension(192),))
('Restoring', 'Mixed_6e/Branch_2/Co

('Restoring', 'Mixed_7c/Branch_2/Conv2d_0a_1x1/weights', (1, 1, Dimension(2048), 448))
('Restoring', 'Mixed_7c/Branch_2/Conv2d_0a_1x1/BatchNorm/beta', (Dimension(448),))
('Restoring', 'Mixed_7c/Branch_2/Conv2d_0a_1x1/BatchNorm/moving_mean', (Dimension(448),))
('Restoring', 'Mixed_7c/Branch_2/Conv2d_0a_1x1/BatchNorm/moving_variance', (Dimension(448),))
('Restoring', 'Mixed_7c/Branch_2/Conv2d_0b_3x3/weights', (3, 3, Dimension(448), 384))
('Restoring', 'Mixed_7c/Branch_2/Conv2d_0b_3x3/BatchNorm/beta', (Dimension(384),))
('Restoring', 'Mixed_7c/Branch_2/Conv2d_0b_3x3/BatchNorm/moving_mean', (Dimension(384),))
('Restoring', 'Mixed_7c/Branch_2/Conv2d_0b_3x3/BatchNorm/moving_variance', (Dimension(384),))
('Restoring', 'Mixed_7c/Branch_2/Conv2d_0c_1x3/weights', (1, 3, Dimension(384), 384))
('Restoring', 'Mixed_7c/Branch_2/Conv2d_0c_1x3/BatchNorm/beta', (Dimension(384),))
('Restoring', 'Mixed_7c/Branch_2/Conv2d_0c_1x3/BatchNorm/moving_mean', (Dimension(384),))
('Restoring', 'Mixed_7c/Branch_2/C

[]

In [16]:
assert(len(tf.trainable_variables()) == 0)

In [17]:
chekpointvars = list(reader.get_variable_to_shape_map().keys())
set(st_variables).difference(set(chekpointvars)), set(chekpointvars).difference(set(st_variables)) 

(set(),
 {'InceptionV3/Conv2d_1a_3x3/BatchNorm/beta',
  'InceptionV3/Conv2d_1a_3x3/BatchNorm/moving_mean',
  'InceptionV3/Conv2d_1a_3x3/BatchNorm/moving_variance',
  'InceptionV3/Conv2d_1a_3x3/weights',
  'InceptionV3/Conv2d_2a_3x3/BatchNorm/beta',
  'InceptionV3/Conv2d_2a_3x3/BatchNorm/moving_mean',
  'InceptionV3/Conv2d_2a_3x3/BatchNorm/moving_variance',
  'InceptionV3/Conv2d_2a_3x3/weights',
  'InceptionV3/Conv2d_2b_3x3/BatchNorm/beta',
  'InceptionV3/Conv2d_2b_3x3/BatchNorm/moving_mean',
  'InceptionV3/Conv2d_2b_3x3/BatchNorm/moving_variance',
  'InceptionV3/Conv2d_2b_3x3/weights',
  'InceptionV3/Conv2d_3b_1x1/BatchNorm/beta',
  'InceptionV3/Conv2d_3b_1x1/BatchNorm/moving_mean',
  'InceptionV3/Conv2d_3b_1x1/BatchNorm/moving_variance',
  'InceptionV3/Conv2d_3b_1x1/weights',
  'InceptionV3/Conv2d_4a_3x3/BatchNorm/beta',
  'InceptionV3/Conv2d_4a_3x3/BatchNorm/moving_mean',
  'InceptionV3/Conv2d_4a_3x3/BatchNorm/moving_variance',
  'InceptionV3/Conv2d_4a_3x3/weights',
  'InceptionV3/Mi

In [18]:
def test_uninitialized(sess):
    global_vars          = tf.global_variables()
    is_not_initialized   = sess.run([tf.is_variable_initialized(var) for var in global_vars])
    not_initialized_vars = [v for (v, f) in zip(global_vars, is_not_initialized) if not f]
    if not_initialized_vars:
        print ('Not initalized varibles are: ', [str(i.name) for i in not_initialized_vars])
        # from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
        # print_tensors_in_checkpoint_file(checkpoint_path, all_tensors=True, tensor_name='')
    assert not not_initialized_vars

In [28]:
with tf.Session() as sess:
#     restore_fn(sess)
    sess.run(tf.global_variables_initializer())
    test_uninitialized(sess)
    const_graph = tf.graph_util.convert_variables_to_constants(sess, tf.get_default_graph().as_graph_def(), [
        'lstm/initial_state',
        "softmax",
        "lstm/state"
    ])

INFO:tensorflow:Froze 382 variables.
Converted 382 variables to const ops.


In [30]:
output_graph = os.path.join(os.path.dirname(checkpoint_path), 'const_' + os.path.basename(checkpoint_path) + '.pb')
with tf.gfile.GFile(output_graph, "wb") as f:
    f.write(const_graph.SerializeToString())

In [32]:
!ls -alh {output_graph} 

-rw-rw-r-- 1 aghasy aghasy 143M Sep 11 14:57 /hdd/models/im2txt_2016_10_05.1000000/const_model.ckpt-1000000.pb
